In [2]:
#Convert bird detector labels into YOLOv5 format and move into new directories

import os
import csv
from PIL import Image

# Function to convert coordinates to YOLO format
def convert_to_yolo_format(image_width, image_height, xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / (2 * image_width)
    y_center = (ymin + ymax) / (2 * image_height)
    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height
    return x_center, y_center, width, height

# Path to the directory containing the CSV files
directory = ""

# Get the list of CSV files in the directory
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Process each CSV file
for csv_file in csv_files:
    file_prefix = os.path.splitext(csv_file)[0]
    train_or_test = file_prefix.split('_')[-1]

    # Read the CSV file
    with open(os.path.join(directory, csv_file), 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row

        # Create a dictionary to store the annotations for each image
        annotations = {}

        # Process each annotation in the CSV file
        for row in reader:
            image_path, xmin, ymin, xmax, ymax, label = row
            if image_path not in annotations:
                annotations[image_path] = []
            annotations[image_path].append((xmin, ymin, xmax, ymax, label))

        # Process each image and its annotations
        for image_path, image_annotations in annotations.items():
            # Load the image to get its dimensions
            image = Image.open(os.path.join(directory, image_path))
            image_width, image_height = image.size

            # Convert each annotation to YOLO format
            yolo_annotations = []
            for xmin, ymin, xmax, ymax, label in image_annotations:
                x_center, y_center, width, height = convert_to_yolo_format(
                    image_width, image_height, float(xmin), float(ymin), float(xmax), float(ymax)
                )
                yolo_annotations.append(f"{label} {x_center} {y_center} {width} {height}")

            # Write the YOLO annotations to a text file
            output_file = f"{directory}/{train_or_test}/{os.path.splitext(image_path)[0]}.txt"
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            with open(output_file, 'w') as outfile:
                outfile.write('\n'.join(yolo_annotations))


In [6]:
#Move images into train and val folders-- the Bird Detector images came with separate train/val label files, but the images were all together in one folder

import os
import shutil

# Path to the directory containing the image files
image_directory = ''

# Path to the directory containing the label files
label_directory = ''

# Create "train" and "test" subdirectories in the image directory
train_directory = os.path.join(image_directory, 'train')
test_directory = os.path.join(image_directory, 'test')
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# Iterate through the label directory
for root, dirs, files in os.walk(label_directory):
    for file in files:
        # Get the label file name without extension
        label_file_name = os.path.splitext(file)[0]

        # Check if the corresponding image file exists
        image_file_path = os.path.join(image_directory, f"{label_file_name}.png")
        if not os.path.isfile(image_file_path):
            continue

        # Determine the destination directory based on the label file's parent directory
        if 'train' in root:
            destination_directory = train_directory
        elif 'test' in root:
            destination_directory = test_directory
        else:
            continue

        try:
            # Move the image file to the destination directory
            shutil.move(image_file_path, os.path.join(destination_directory, f"{label_file_name}.jpg"))
        except (shutil.Error, IOError) as e:
            print(f"Failed to move {image_file_path}: {e}")
            continue


Failed to move D:\YOLOv5Rowan\data\birddetector\seabirds_rgb_572.png: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\YOLOv5Rowan\\data\\birddetector\\seabirds_rgb_572.png'
